# MLP

In [1]:
import numpy as np
import time
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt

In [6]:
class PerceptronMulticapa:
    def __init__(self, capas, alpha=0.1):
        self.capas = capas
        self.alpha = alpha
        self.bias = []
        self.pesos = []
        for i in range(0, len(capas) - 1):
            # Inicializar los pesos y bias de cada capa
            peso = np.random.randn(capas[i], capas[i+1])
            self.pesos.append(peso)
            bias = np.zeros(capas[i+1])
            self.bias.append(bias)

    def activacion(self, x):
        # Función de activación sigmoide
        return np.tanh(x)

    def activacion_derivada(self, x):
        # Derivada de la función de activación sigmoide
        return  1 - x ** 2

    def feedforward(self, X):
        # Calcular la salida de cada capa
        capa_activacion = [X]
        for i in range(len(self.capas) - 1):
            x = np.dot(capa_activacion[i], self.pesos[i]) + self.bias[i]
            y = self.activacion(x)
            capa_activacion.append(y)
        return capa_activacion

    def backpropagation(self, X, y, capa_activacion):
        # Calcular el error de la capa de salida
        delta = (capa_activacion[-1] - y) * self.activacion_derivada(capa_activacion[-1])
        deltas = [delta]

        for i in reversed(range(1, len(self.capas) - 1)):
            delta = np.dot(deltas[0], self.pesos[i].T) * self.activacion_derivada(capa_activacion[i])
            deltas.insert(0, delta) 
        
        # Propagar el error hacia atrás a través de la red neuronal
        for i in range(len(self.pesos)):
            a = capa_activacion[i].reshape(-1, 1)
            d = deltas[i].reshape(1, -1)
            self.pesos[i] -= self.alpha * np.dot(a, d)
            self.bias[i] -= self.alpha * deltas[i]

    def entrenar(self, X, y, epochs):
        inicio = time.time()
        for epoch in range(epochs):
            for i in range(len(X)):
                # Feedforward
                capa_activacion = self.feedforward(X[i])

                # Backpropagation
                self.backpropagation(X[i], y[i], capa_activacion)
            if (epoch+1) % 10 == 0:
                print(f"Epoch {epoch+1}/{epochs} completado")
        fin = time.time()
        return fin - inicio

    def predecir(self, X):
        # Obtener la salida de la última capa
        capa_activacion = self.feedforward(X)
        return capa_activacion[-1]
    

## Cargar MNIST

In [3]:
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data.astype(np.float32)
y = mnist.target.astype(int)

# Normalizar a rango [0,1]
X /= 255.0


## Feature Engineering

In [7]:
# Estandarizar los datos (media 0, varianza 1)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# One-hot encoding de las etiquetas
Y = np.eye(10)[y]

# Dividir en entrenamiento y prueba
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(
    X, Y, test_size=0.2, random_state=42, stratify=y
)


## Crear y entrenar la red

In [10]:
perceptron = PerceptronMulticapa(capas=[784, 128, 64, 10], alpha=0.01)
tiempo_entrenamiento = perceptron.entrenar(X_entrenamiento, y_entrenamiento, epochs=50)

# Evaluar con un subconjunto (por tiempo)
predicciones = []
y_reales = []

for i in range(500):
    salida = perceptron.predecir(X_prueba[i])
    pred_clase = np.argmax(salida)
    real_clase = np.argmax(y_prueba[i])
    predicciones.append(pred_clase)
    y_reales.append(real_clase)

Epoch 10/50 completado
Epoch 20/50 completado
Epoch 30/50 completado
Epoch 40/50 completado
Epoch 50/50 completado


## Metricas

In [11]:
accuracy = accuracy_score(y_reales, predicciones)
precision = precision_score(y_reales, predicciones, average='macro')
recall = recall_score(y_reales, predicciones, average='macro')
f1 = f1_score(y_reales, predicciones, average='macro')

print("\n📊 Resultados del modelo:")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-score:  {f1:.4f}")
print(f"Tiempo de entrenamiento: {tiempo_entrenamiento:.2f} segundos")


📊 Resultados del modelo:
Accuracy:  0.7600
Precision: 0.7692
Recall:    0.7552
F1-score:  0.7531
Tiempo de entrenamiento: 528.41 segundos


# MLP con Sklearn

In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [20]:
mlp = MLPClassifier(hidden_layer_sizes=(10, 10), activation= 'tanh', max_iter=10000, random_state=42, solver="sgd")
mlp.fit(X_entrenamiento, y_entrenamiento)

y_pred = mlp.predict(X_prueba)

print("Predicciones: ", y_pred)
print("Valor real  : ",y_prueba)

accuracy = accuracy_score(y_prueba, y_pred)
print("Precisión: {:.2f}".format(accuracy))

confusion_matrix(y_prueba,y_pred)

Predicciones:  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Valor real  :  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Precisión: 1.00


array([[10,  0,  0],
       [ 0, 10,  0],
       [ 0,  0, 10]], dtype=int64)